In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [15]:
from fastai.imports import *
from fastai.torch_imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from sklearn import metrics
tqdm.monitor_interval = 0

In [3]:
torch.cuda.is_available()
torch.backends.cudnn.enabled

True

In [4]:
PATH = 'data/dogbreeds/'
sz = 224
arch=resnext101_64 #or 34
bs = 58

In [5]:
label_csv = f'{PATH}labels.csv'
n = len(list(open(label_csv))) - 1
val_idxs = get_cv_idxs(n) #takes default (0.2?) amount of indices from 0 to n from training set for val

In [6]:
label_df = pd.read_csv(label_csv)
label_df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [50]:
tfms = tfms_from_model(arch, sz, aug_tfms = transforms_side_on, max_zoom = 1.1)
data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv', test_name = 'test', 
                                    val_idxs = val_idxs, suffix = '.jpg', tfms = tfms, bs = bs)
fn = PATH+data.trn_ds.fnames[0]; fn

'data/dogbreeds/train/001513dfcb2ffafc82cccf4d8bbaba97.jpg'

In [7]:
def get_data(sz, bs):
    tfms = tfms_from_model(arch, sz, aug_tfms = transforms_side_on, max_zoom = 1.1)
    data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv', test_name = 'test', num_workers = 4,
                                        val_idxs = val_idxs, suffix = '.jpg', tfms = tfms, bs = bs)
    #return data if sz > 300 else data.resize(340, 'tmp')
    return data

In [8]:
data = get_data(sz, bs)
learn = ConvLearner.pretrained(arch, data, precompute = True, ps = 0.5)
learn.fit(1e-2, 3)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                   
    0      1.121323   0.399277   0.916667  
    1      0.508793   0.284693   0.920498                    
    2      0.355476   0.245375   0.928161                    



[0.24537516, 0.9281609240505431]

In [9]:
learn.precompute = False
learn.fit(1e-2, 3, cycle_len = 1)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                    
    0      0.346643   0.233042   0.931992  
    1      0.331913   0.228431   0.93295                     
    2      0.307184   0.222452   0.931992                    



[0.22245179, 0.9319923371076584]

In [12]:
learn.set_data(get_data(299, bs))
learn.fit(1e-2, 3, cycle_len = 1)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                    
    0      0.307655   0.215376   0.931992  
    1      0.292355   0.209907   0.933908                    
    2      0.273383   0.201704   0.937261                    



[0.2017036, 0.9372605366839303]

In [13]:
learn.fit(1e-2, 3, cycle_len = 1, cycle_mult = 2) #try doing this before the previous cell
#But because val stops decreasing, should do the size expansion first. If no other way to decrease val,
#then can do cycle mult to lower training error

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                    
    0      0.259306   0.19887    0.934866  
    1      0.238372   0.200806   0.936303                    
    2      0.229105   0.192455   0.940134                    
    3      0.238044   0.202879   0.935345                    
    4      0.217512   0.193807   0.933429                    
    5      0.194446   0.194379   0.937261                    
    6      0.194479   0.193128   0.937739                    



[0.19312826, 0.9377394649717543]

In [1]:
#learn.save('regular')
learn.load('regular')
learn.set_data(get_data(350, 14))
#lrf = learn.lr_find()
#learn.sched.plot()
learn.fit(1e-2, 1, cycle_len = 1)

NameError: name 'learn' is not defined

In [24]:
log_preds, y = learn.TTA(is_test = True)
probs = np.mean(np.exp(log_preds), 0)
#accuracy(np.mean((log_preds),0), y), metrics.log_loss(y, probs)

TypeError: torch.max received an invalid combination of arguments - got (numpy.ndarray, dim=int), but expected one of:
 * (torch.FloatTensor source)
 * (torch.FloatTensor source, torch.FloatTensor other)
      didn't match because some of the keywords were incorrect: dim
 * (torch.FloatTensor source, int dim)
 * (torch.FloatTensor source, int dim, bool keepdim)


In [27]:
probs.shape
submit = pd.DataFrame(probs)
submit.columns = data.classes
submit.insert(0, 'id', [Id[5:-4] for Id in data.test_ds.fnames]) #skip prefix and suffix 5:-4

In [28]:
SUBM = f'{PATH}subm/'
os.makedirs(SUBM, exist_ok = True)
submit.to_csv(f'{SUBM}submit.csv', index = False)
FileLink(f'{SUBM}submit.csv')

/home/Archimedes/fastai/courses/dl1/data/dogbreeds/subm/submit.csv